# Predicting Top Genre Given A Year

### Merge clustered and billboard dataset

### Feature Engineering

Process of creating new features like columns from our data to make the model for effective and accurate at learning patterns 


example -> ratios for distinguishing upbeat vs aggressive music or like energy x danceability predicting a specific genre

### Split, Train, and Test Data

Training data could be -> 1980 - 2009

Testing data could be -> 2010 - 2019

### Linear Regression

### Predictions

### Evalutation
MSE, MAE, Precision, Recall

### Visualizations & Conclusion